# RAG Menggunakan Layanan Agen Azure AI & Semantic Kernel

Cuplikan kode ini menunjukkan cara membuat dan mengelola agen Azure AI untuk retrieval-augmented generation (RAG) menggunakan `Azure AI Agent Service` dan `Semantic Kernel`. Agen ini memproses pertanyaan pengguna berdasarkan konteks yang diambil dan memberikan respons yang akurat sesuai.


Perbaikan Versi SQLite  
Jika Anda mengalami kesalahan:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Hapus komentar pada blok kode ini di awal notebook Anda:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Mengimpor Paket
Kode berikut mengimpor paket-paket yang diperlukan:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Retrieval-Augmented Generation dengan Semantic Kernel & Azure AI Agent Service

Contoh ini menunjukkan cara menggunakan **Azure AI Agent Service** untuk melakukan **Retrieval-Augmented Generation (RAG)** dengan menggabungkan model bahasa dengan konteks spesifik domain dari dokumen yang diunggah.

### Cara Kerjanya

1. **Unggah Dokumen**: File markdown (document.md) yang berisi informasi (kebijakan asuransi perjalanan Contoso) diunggah ke layanan agen.

2. **Pembuatan Vector Store**: Dokumen diindeks ke dalam vector store untuk memungkinkan pencarian semantik atas isinya.

3. **Konfigurasi Agen**: Agen dibuat menggunakan model `gpt-4o` dengan instruksi ketat berikut:
   - Hanya menjawab pertanyaan berdasarkan konten yang diambil dari dokumen.
   - Menolak menjawab jika pertanyaan berada di luar cakupan.

4. **Integrasi File Search Tool**: `FileSearchTool` didaftarkan ke agen, memungkinkan model untuk mencari dan mengambil cuplikan relevan dari dokumen yang diindeks selama inferensi.

5. **Interaksi Pengguna**: Pengguna dapat mengajukan pertanyaan. Jika informasi relevan ditemukan dalam dokumen, agen menghasilkan jawaban yang terarah.  
   Jika tidak, agen secara eksplisit menjawab bahwa dokumen tidak mengandung informasi yang cukup.


### Fungsi Utama

Fungsi utama dalam program ini bertanggung jawab untuk mengatur alur eksekusi. Pastikan untuk memahami setiap langkah sebelum melakukan modifikasi.

```python
def main():
    # Inisialisasi variabel dan pengaturan awal
    initialize_settings()

    # Memulai proses utama
    start_process()

    # Menangani hasil akhir
    handle_results()
```

### Langkah-Langkah

1. **Inisialisasi**  
   Pastikan semua pengaturan awal sudah benar sebelum melanjutkan. Ini termasuk memuat konfigurasi dan memeriksa dependensi.

2. **Proses Utama**  
   Jalankan logika utama program di sini. Pastikan untuk menangani semua kemungkinan skenario.

3. **Hasil Akhir**  
   Setelah proses selesai, pastikan untuk memproses hasil dengan benar, seperti menyimpan data atau memberikan output kepada pengguna.

### Catatan Tambahan

- Jangan lupa untuk menambahkan log di setiap langkah penting agar mudah untuk debugging.
- Jika terjadi kesalahan, pastikan untuk menangani exception dengan baik agar program tidak berhenti secara tiba-tiba.


Pastikan untuk terlebih dahulu menjalankan `az login` menggunakan Azure CLI sehingga konteks autentikasi yang tepat disediakan saat menggunakan `DefaultAzureCredential`. Azure AI Agent Service tidak menggunakan kunci API.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Silakan berikan file markdown yang ingin diterjemahkan, dan saya akan menerjemahkannya sesuai dengan aturan yang telah Anda tetapkan.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berusaha untuk memberikan hasil yang akurat, harap diketahui bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang otoritatif. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
